In [1]:
from requests import Session
from tabulate import tabulate
import json
import io
session = Session()
tokenResponse = session.post('https://api.etilbudsavis.dk/v2/sessions?api_key=00izwus560vpbdasf0o92m2fhaim4ynr')
token = (json.loads(tokenResponse.text))["token"]
if not token:
    print "Couldn't get token"
defaultHeaders = {'X-token': token, 'Origin': "https://etilbudsavis.dk/"}

In [2]:
#List Store. 
listResponse = session.get('https://api.etilbudsavis.dk/v2/stores?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33&limit=100',
                              headers=defaultHeaders)
stores = json.loads(listResponse.text)
storeList = [[store['branding']['name'], store['id']] for store in stores]
print "-----------------List 100 stores-----------------------------"
print tabulate(storeList, headers=['All Stores', 'Id'])
#Should use the branding name instead of store id to filter
#selectedId = ['7edbLOE', '90f2F7y', 'd8dewIy', '2665CXz', '82a9zXz', 'b88bQ7y', '618fq21', '98c86Iq']
#selectedStoreList = [[store['branding']['name'], store['id']] for store in stores if (store['id'] in selectedId)]
#print tabulate(selectedStoreList, headers=['Selected Stores', 'Id'])

#Note: input names in lower-case
selectedBrandNames = ['coop', 'kiwi', 'meny', 'rema', 'matkroken', 'joker', 'bunnpris', 'extra']


-----------------List 100 stores-----------------------------
All Stores              Id
----------------------  -------
Matkroken Tåsen         7edbLOE
Apotek 1                d146811
SPAR                    2a7dV81
Norli                   ec06Mm3
Christiania Belysning   356ePLq
KIWI                    90f2F7y
MENY                    d8dewIy
Coop Mega Nydalen       2665CXz
Apotek 1                77f6011
Vitus Apotek            57e1mVq
Apotek 1                49b2Gy1
BilXtra                 36346GD
Jernia                  84d5SND
Sport1                  188aLDq
Barnas Hus              acefKMD
Tilbords                bcd4GjA
Life Helsekost          4d08O6D
Vita                    ef73ozq
G-Sport Oslo - Ullevål  d1b6OK3
Coop Mega Ullevål       82a9zXz
KIWI                    b88bQ7y
Elkjøp                  de03l41
Närbutikken             67f3Wvq
Joker                   618fq21
REMA 1000               6d3dpaz
REMA 1000               793fTYz
Joker                   00f0A21
Plantasjen     

In [3]:
#List Catalog
#The store_ids filter of the API does not seem to work, so we have to filter ourself!
#storeIdStr = ','.join([selectedStore[1] for selectedStore in selectedStoreList])
catalogsResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33&limit=100',
                               headers=defaultHeaders)
catalogs = json.loads(catalogsResponse.text)
catalogList = [[catalog['branding']['name'], catalog['store_id'], catalog['id']] for catalog in catalogs]
print tabulate(catalogList, headers=['All Catalogs', 'Store Id', 'Id'])

#Check if any the brand name contains any selectedBrandName
selectedCatalogs = [catalog for catalog in catalogs 
                    if len([brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower()]) > 0]
selectedCatalogList = [[catalog['branding']['name'], catalog['id'], catalog['run_from']] for catalog in selectedCatalogs]
print "\n---------------------------------"
print tabulate(selectedCatalogList, headers=['Selected Catalogs', 'Id', 'Run From'])

All Catalogs               Store Id    Id
-------------------------  ----------  -------
Jula                       12d0Ta3     89c2hJq
Nille                      0659dF3     3497UJq
Rusta                      2d99HSq     1f0cTJq
Jernia                     84d5SND     1742ENq
KIWI                       1df9fZ8     80005Jq
Skeidar                    4e7fMkD     5991IJq
Extra Bekkestua            98c86Iq     70falTq
Montér                     acc2IOD     0467hNq
Biltema                    d026SA1     d35c32q
Bunnpris Nydalen           f00d501     ea96pdq
Bohus                      0f62X4A     39afSVq
Søstrene Grene             55d6pg1     2459mxq
BAUHAUS                    8dca4Wy     f922pJq
Elektroimportøren          deb3p4A     d545CNq
BYGGI                      59b49TE     0e84XNq
Biltema                    d026SA1     a6e0ZYA
Kjell & Company            ac8cvjD     17aaCdq
Panduro Hobby              1205lK1     bdf4rLq
Coop Obs! Bygg Alnabru     59e3rrA     9b14HTq
Jacobs            

In [15]:
#Get offers in selected catalog
for catalogStore, catalogId, catalogRunFrom in selectedCatalogList:
    print "--------------------------------------------"
    print "Requesting catalog "+catalogStore+" with id "+catalogId+"..."
    offersResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs/'+catalogId+'/hotspots?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33',
                        headers=defaultHeaders)
    offers = json.loads(offersResponse.text)
    print "Number of offers: "+ str(len(offers))
    if len(offers)==0:
        continue
    offerIdList = [offer['id'] for offer in offers]
    print "Parsing offers..."
    JsonFileName = catalogStore + catalogRunFrom + ".json"
    offerDescList = []
    for offerId in offerIdList:
        offerDescResponse = session.get('https://api.etilbudsavis.dk/v2/offers/'+offerId+'?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33',
                                    headers=defaultHeaders)
        offerDescList.append(json.loads(offerDescResponse.text.encode('utf-8')))
    with io.open(JsonFileName, 'w', encoding='utf-8') as json_file:
        data = json.dumps(offerDescList, ensure_ascii=False)
        # unicode(data) auto-decodes data to unicode if str
        json_file.write(unicode(data))
        
    print "Done."

--------------------------------------------
Requesting catalog KIWI with id 80005Jq...
Number of offers: 86
Parsing offers...
Done.
--------------------------------------------
Requesting catalog Extra Bekkestua with id 70falTq...
Number of offers: 24
Parsing offers...
Done.
--------------------------------------------
Requesting catalog Bunnpris Nydalen with id ea96pdq...
Number of offers: 0
--------------------------------------------
Requesting catalog Coop Obs! Bygg Alnabru with id 9b14HTq...
Number of offers: 48
Parsing offers...
Done.
--------------------------------------------
Requesting catalog Coop Obs! Bygg Lillestrøm with id 21e8tTq...
Number of offers: 48
Parsing offers...


UnicodeEncodeError: 'ascii' codec can't encode character u'\xf8' in position 23: ordinal not in range(128)

In [16]:
print u'xf8'

xf8
